In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

import tkinter as tk
from tkinter import messagebox

c:\Users\Home\anaconda3\envs\DLassignment2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load your dataset
df = pd.read_csv(r'D:\NLP_Assignments\23I-8038_NLP_A4\Text Summarization Data.csv')


In [4]:
# Display column names and number of features
print("Columns in the dataset:")
print(df.columns)

print("\nNumber of features (columns):", len(df.columns))

Columns in the dataset:
Index(['id', 'article', 'highlights'], dtype='object')

Number of features (columns): 3


In [5]:
# We only need the 'article' and 'highlights' columns
df = df[['article', 'highlights']]

In [6]:
# Drop rows with missing data
df.dropna(inplace=True)

In [7]:
# Reset index after dropping rows
df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

(13368, 2)

In [8]:
df = df.rename(columns={'article': 'text', 'highlights': 'summary'})

In [9]:
df.head()

,text,summary
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...


In [10]:
# Tokenizer and Model setup
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
# Freeze encoder layers for faster training
for param in model.encoder.parameters():
    param.requires_grad = False

In [12]:
# Tokenization function
def tokenize_data(df, max_input_length=256, max_target_length=64):
    inputs = tokenizer(df['text'].tolist(), padding=True, truncation=True, max_length=max_input_length, return_tensors='pt')
    targets = tokenizer(df['summary'].tolist(), padding=True, truncation=True, max_length=max_target_length, return_tensors='pt')
    return inputs, targets

In [ ]:
# Split dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [14]:
train_inputs, train_targets = tokenize_data(train_df)
val_inputs, val_targets = tokenize_data(val_df)

In [15]:
# Custom dataset class for PyTorch DataLoader
class SummarizationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.targets['input_ids'][idx]
        }

In [16]:
# Create DataLoader
train_dataset = SummarizationDataset(train_inputs, train_targets)
val_dataset = SummarizationDataset(val_inputs, val_targets)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4)


In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10, 
    weight_decay=0.01,
    fp16=True,  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

c:\Users\Home\anaconda3\envs\DLassignment2\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.078600,1.859972
2,2.029000,1.845963
3,1.977800,1.844048
4,1.942200,1.839024
5,1.958100,1.834136
6,1.956600,1.829594
7,1.915000,1.833622
8,1.912000,1.833661
9,1.884800,1.833395
10,1.893600,1.831903


TrainOutput(global_step=30080, training_loss=1.947475141667305, metrics={'train_runtime': 2157.72, 'train_samples_per_second': 55.758, 'train_steps_per_second': 13.941, 'total_flos': 8141486067548160.0, 'train_loss': 1.947475141667305, 'epoch': 10.0})

In [ ]:
model.save_pretrained('./fine_tuned_t5')
tokenizer.save_pretrained('./fine_tuned_t5')

('./fine_tuned_t5/tokenizer_config.json',
 './fine_tuned_t5/special_tokens_map.json',
 './fine_tuned_t5/spiece.model',
 './fine_tuned_t5/added_tokens.json')

In [68]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./fine_tuned_t5')
tokenizer = T5Tokenizer.from_pretrained('./fine_tuned_t5')


In [69]:

# Function to summarize text
def summarize_text(text, max_length=150, num_beams=4):
    # Preprocess the input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the summary with beam search
    summary_ids = model.generate(inputs, max_length=max_length, num_beams=num_beams, early_stopping=True)

    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

# Example input text (you can replace it with your own)
input_text = """
Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from and make decisions based on data.
It involves the use of algorithms and statistical models to analyze patterns in data and make predictions or decisions without being explicitly programmed.
Machine learning has become a significant area of research and application across various industries, from healthcare to finance, enabling systems to improve their performance over time through experience.
The primary goal of machine learning is to develop algorithms that can generalize from previous observations, allowing them to perform well on unseen data.
"""

# Get the summary
summary = summarize_text(input_text)

print("Original Text:")
print(input_text)
print("\nSummary:")
print(summary)


Original Text:

Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. 
It involves the use of algorithms and statistical models to analyze patterns in data and make predictions or decisions without being explicitly programmed. 
Machine learning has become a significant area of research and application across various industries, from healthcare to finance, enabling systems to improve their performance over time through experience. 
The primary goal of machine learning is to develop algorithms that can generalize from previous observations, allowing them to perform well on unseen data.


Summary:
Machine learning is a branch of artificial intelligence that focuses on building systems that can learn from and make decisions based on data. It involves the use of algorithms and statistical models to analyze patterns in data and make predictions or decisions without being explicitly programmed.
